In [ ]:
!pip install transformers

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

url = "https://raw.githubusercontent.com/banglanlp/bnlp-resources/main/sentiment/ABSA_datasets/BASA_cricket_"

data_files = {
    "train" : url + "train.tsv",
    "test" : url + "test.tsv"
}

bangla_cricket_dataset = load_dataset("csv", data_files = data_files, sep = "\t")


Using custom data configuration default-f8b1169f8fffa4b2
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-f8b1169f8fffa4b2/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e)


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
bangla_cricket_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'class_label'],
        num_rows: 2085
    })
    test: Dataset({
        features: ['id', 'text', 'class_label'],
        num_rows: 380
    })
})

In [ ]:
bangla_cricket_dataset["train"][14]

{'class_label': 'negative',
 'id': 227,
 'text': 'সংবাদ সমল্মেনে এসে জাতীয় বীর ম্যাশের সহজ কথা- আমরা খেলেছি বোলিং পিচে কিন্তু তারা খেলেছে ব্যাটিং পিচে।'}

### Initializing tokenizer and datacollator

In [ ]:
from transformers import AutoTokenizer, DataCollatorWithPadding

checkpoint = "monsoon-nlp/bangla-electra"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer

loading configuration file https://huggingface.co/monsoon-nlp/bangla-electra/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ed4bba197748161af54c3248a26f97ded9e3509fbefda78285ececa1123fcbe3.c8f2f21ce33891d47310cad825c570e6b77f1cbd2fecde724b1e7bdd2080c0f3
Model config ElectraConfig {
  "_name_or_path": "monsoon-nlp/bangla-electra",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

PreTrainedTokenizerFast(name_or_path='monsoon-nlp/bangla-electra', vocab_size=29898, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer)
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='monsoon-nlp/bangla-electra', vocab_size=29898, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [ ]:
def tokenize_function(example):
  return tokenizer(
      example["text"],
      truncation = True
  )

tokenized_datasets = bangla_cricket_dataset.map(tokenize_function, batched = True)
tokenized_datasets

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-f8b1169f8fffa4b2/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-dd04acaa5d5083c0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-f8b1169f8fffa4b2/0.0.0/6b9057d9e23d9d8a2f05b985917a0da84d70c5dae3d22ddd8a3f22fb01c69d9e/cache-1ea8f0be51311f81.arrow


DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'class_label', 'id', 'input_ids', 'text', 'token_type_ids'],
        num_rows: 2085
    })
    test: Dataset({
        features: ['attention_mask', 'class_label', 'id', 'input_ids', 'text', 'token_type_ids'],
        num_rows: 380
    })
})

In [ ]:
tokenized_datasets["train"][0]

{'attention_mask': [1, 1, 1, 1, 1, 1, 1],
 'class_label': 'neutral',
 'id': 2760,
 'input_ids': [3, 18913, 2386, 2288, 3728, 35, 4],
 'text': 'আফতাব আপনি ভালো আছেন?',
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0]}

In [ ]:
type(tokenized_datasets)

datasets.dataset_dict.DatasetDict

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("test-trainer")


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
model.config

loading configuration file https://huggingface.co/monsoon-nlp/bangla-electra/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/ed4bba197748161af54c3248a26f97ded9e3509fbefda78285ececa1123fcbe3.c8f2f21ce33891d47310cad825c570e6b77f1cbd2fecde724b1e7bdd2080c0f3
Model config ElectraConfig {
  "_name_or_path": "monsoon-nlp/bangla-electra",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "voc

ElectraConfig {
  "_name_or_path": "monsoon-nlp/bangla-electra",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 128,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "electra",
  "num_attention_heads": 4,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "summary_activation": "gelu",
  "summary_last_dropout": 0.1,
  "summary_type": "first",
  "summary_use_proj": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 29898
}

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model, 
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["test"],
    data_collator = data_collator, 
    tokenizer = tokenizer, 
)

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `ElectraForSequenceClassification.forward` and have been ignored: id, text, class_label.
***** Running training *****
  Num examples = 2085
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 783


KeyError: ignored

In [ ]:
!wget https://www.kaggle.com/furcifer/bangla-newspaper-dataset/download